## Deeplearning Classifier

In [33]:
# Import necessary libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

In [12]:
# Load the dataset
df = pd.read_csv('Symptom2Disease.csv')
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [14]:
num_categories = df['label'].nunique()
print("The number of categories of diseases are:", num_categories)

The number of categories of diseases are: 24


In [15]:
# Split the dataset into input (text) and output (label) columns
text_data = df['text'].values
labels = df['label'].values

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [16]:
# Split the dataset into training and testing sets
text_train, text_test, labels_train, labels_test = train_test_split(text_data, labels, test_size=0.2, random_state=42)

# Create a tokenizer and fit it on the training text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

# Convert the text data to sequences of tokens
sequences_train = tokenizer.texts_to_sequences(text_train)
sequences_test = tokenizer.texts_to_sequences(text_test)

# Pad the sequences to have the same length
max_sequence_length = max(len(seq) for seq in sequences_train)
sequences_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Determine the number of classes
num_classes = len(label_encoder.classes_)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Add L2 regularization
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(sequences_train, labels_train, epochs=50, batch_size=32)

Epoch 1/50
30/30 [==============================] - 3s 31ms/step - loss: 4.2161 - accuracy: 0.0396
Epoch 2/50
30/30 [==============================] - 1s 32ms/step - loss: 3.7872 - accuracy: 0.0542
Epoch 3/50
30/30 [==============================] - 1s 32ms/step - loss: 3.5297 - accuracy: 0.0531
Epoch 4/50
30/30 [==============================] - 1s 38ms/step - loss: 3.3720 - accuracy: 0.0385
Epoch 5/50
30/30 [==============================] - 1s 35ms/step - loss: 3.2695 - accuracy: 0.0458
Epoch 6/50
30/30 [==============================] - 1s 36ms/step - loss: 3.1727 - accuracy: 0.0750
Epoch 7/50
30/30 [==============================] - 1s 32ms/step - loss: 3.0719 - accuracy: 0.1031
Epoch 8/50
30/30 [==============================] - 1s 32ms/step - loss: 2.9231 - accuracy: 0.1292
Epoch 9/50
30/30 [==============================] - 1s 33ms/step - loss: 2.7338 - accuracy: 0.1677
Epoch 10/50
30/30 [==============================] - 1s 29ms/step - loss: 2.5421 - accuracy: 0.2083
Epoch 11/

In [36]:
_, accuracy = model.evaluate(sequences_test, labels_test)
print("Accuracy:", accuracy)

8/8 [==============================] - 0s 7ms/step - loss: 0.5780 - accuracy: 0.8708
Accuracy: 0.8708333373069763


In [34]:
# Predict on the test set
predictions = model.predict(sequences_test)
predicted_labels = np.argmax(predictions, axis=1)

# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Generate the classification report
report = classification_report(label_encoder.inverse_transform(labels_test), predicted_labels)
print(report)


8/8 [==============================] - 0s 6ms/step
                                 precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00         7
                      Arthritis       1.00      1.00      1.00        10
               Bronchial Asthma       1.00      1.00      1.00        11
           Cervical spondylosis       1.00      1.00      1.00         7
                    Chicken pox       0.55      0.50      0.52        12
                    Common Cold       1.00      0.83      0.91        12
                         Dengue       0.56      0.75      0.64        12
          Dimorphic Hemorrhoids       1.00      1.00      1.00         7
               Fungal infection       1.00      0.85      0.92        13
                   Hypertension       1.00      1.00      1.00        10
                       Impetigo       0.83      0.91      0.87        11
                       Jaundice       1.00      1.00      1.00        11

In [37]:
# Create a function for prediction
def predict_disease(text):
    # Load the trained model
    model = tf.keras.models.load_model('disease_model.h5')

    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_sequence_length)

    # Make the prediction
    predicted_probs = model.predict(sequence)[0]
    predicted_class = tf.argmax(predicted_probs).numpy()
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    # Calculate the confidence score
    predicted_prob = predicted_probs[predicted_class]

    # Return the predicted class and confidence score
    return predicted_label, predicted_prob

In [101]:
# Test the predict_disease function
input_text = "My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."
predicted_class = predict_disease(input_text)
print(predicted_class)

1/1 [==============================] - 0s 51ms/step
('Psoriasis', 0.9839914)


## Random Forest Classifier

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [23]:
# Read the data from the CSV file
data = pd.read_csv('Symptom2Disease.csv')

In [24]:
data.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [28]:

# Extract symptom texts and disease names from the dataframe
symptoms = data['text'].tolist()
diseases = data['label'].tolist()

# Convert symptom texts into numerical feature vectors using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(symptoms)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, diseases, test_size=0.2, random_state=42)

# Initialize a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = rf_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9708333333333333


In [29]:
# Calculate precision, recall, and F1-score
classification_report = classification_report(y_test, y_pred, digits=4)
print("Classification Report:\n", classification_report)

Classification Report:
                                  precision    recall  f1-score   support

                           Acne     1.0000    1.0000    1.0000         7
                      Arthritis     1.0000    1.0000    1.0000        10
               Bronchial Asthma     1.0000    1.0000    1.0000        11
           Cervical spondylosis     1.0000    1.0000    1.0000         7
                    Chicken pox     0.8462    0.9167    0.8800        12
                    Common Cold     1.0000    0.9167    0.9565        12
                         Dengue     0.9000    0.7500    0.8182        12
          Dimorphic Hemorrhoids     1.0000    1.0000    1.0000         7
               Fungal infection     1.0000    1.0000    1.0000        13
                   Hypertension     1.0000    1.0000    1.0000        10
                       Impetigo     1.0000    1.0000    1.0000        11
                       Jaundice     1.0000    1.0000    1.0000        11
                        Ma

In [51]:
# Export the trained model to a file
import pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(rf_classifier, file)

# Function to predict disease label from input text
def predict_disease(text):
    # Load the saved model
    with open(model_filename, 'rb') as file:
        model = pickle.load(file)
    
    # Transform the input text into numerical features
    input_vector = vectorizer.transform([text])
    
    # Predict the disease label
    predicted_label = model.predict(input_vector)
    confidence_scores = model.predict_proba(input_vector)
    return predicted_label[0], confidence_scores[0][predicted_label[0]]

In [ ]:
predict_disease("I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.")